<a href="https://colab.research.google.com/github/kevinshiroya123/DL-project/blob/main/234_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycryptodome # Install the correct package using pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00


In [ ]:
import os
import random
import string
import hashlib
from Crypto.Cipher import AES

# ---------------- STEP 1: Create a Sample Text File ----------------
def create_sample_text_file(filename, text):
    """Creates a text file with given text."""
    with open(filename, 'w') as file:
        file.write(text)
    print(f"✅ Sample text file '{filename}' created!")

# ---------------- STEP 2: Generate Random Encryption Key & IV ----------------
def generate_random_key(length=32):
    """Generates a secure random encryption key."""
    return ''.join(random.choices(string.ascii_letters + string.digits + "QWERTYUIOPASDFGHJKLZXCVBNM1234567890!@#$%^&*", k=length))

def generate_random_iv():
    """Generates a 16-byte random IV for AES encryption."""
    return os.urandom(16)

# ---------------- STEP 3: Encrypt the File ----------------
def encrypt_file(input_filename):
    """Encrypts a file using AES-256 CBC encryption."""
    if not os.path.exists(input_filename):
        print(f"❌ Error: File '{input_filename}' not found!")
        return None

    with open(input_filename, 'rb') as file:
        data = file.read()

    # Step 1: Generate Random Key & IV
    encryption_key = generate_random_key()
    iv = generate_random_iv()

    # Step 2: Convert key into a 256-bit hash
    key_hash = hashlib.sha256(encryption_key.encode()).digest()

    # Step 3: Ensure data is padded to a multiple of 16 bytes
    padding_length = 16 - (len(data) % 16)
    padded_data = data + bytes([padding_length]) * padding_length

    # Step 4: Encrypt using AES-256 CBC mode
    cipher = AES.new(key_hash, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(padded_data)

    # Step 5: Save encrypted file (IV + ciphertext)
    encrypted_filename = input_filename + ".enc"
    with open(encrypted_filename, 'wb') as file:
        file.write(iv + ciphertext)

    print(f"✅ File '{input_filename}' encrypted and saved as '{encrypted_filename}'!")
    return encrypted_filename, encryption_key  # Return filename & key for decryption

# ---------------- STEP 4: Decrypt the File ----------------
def decrypt_file(encrypted_filename, encryption_key):
    """Decrypts an AES-256 CBC encrypted file and restores original content."""
    if not os.path.exists(encrypted_filename):
        print(f"❌ Error: File '{encrypted_filename}' not found!")
        return None

    # Step 1: Convert key into 256-bit hash
    key_hash = hashlib.sha256(encryption_key.encode()).digest()

    with open(encrypted_filename, 'rb') as file:
        iv = file.read(16)  # Extract IV
        ciphertext = file.read()

    # Step 2: Decrypt the file
    cipher = AES.new(key_hash, AES.MODE_CBC, iv)
    decrypted_padded_data = cipher.decrypt(ciphertext)

    # Step 3: Remove padding
    padding_length = decrypted_padded_data[-1]
    decrypted_data = decrypted_padded_data[:-padding_length]

    # Step 4: Save decrypted file
    decrypted_filename = encrypted_filename.replace(".enc", "_decrypted.txt")
    with open(decrypted_filename, 'wb') as file:
        file.write(decrypted_data)

    print(f"✅ File '{encrypted_filename}' decrypted and saved as '{decrypted_filename}'!")
    return decrypted_filename

# ---------------- RUN ENCRYPTION & DECRYPTION ----------------
# Step 1: Create a sample text file
sample_text = "This is a simple encryption test. Let's see if it works correctly!"
text_filename = "test.txt"
create_sample_text_file(text_filename, sample_text)

# Step 2: Encrypt the file
encrypted_file, key = encrypt_file(text_filename)

# Step 3: Decrypt the file using the same key
if encrypted_file:
    decrypted_file = decrypt_file(encrypted_file, key)

    # Step 4: Verify if decryption worked
    with open(decrypted_file, 'r') as file:
        decrypted_content = file.read()
    print("\n🔍 Decrypted Content:")
    print(decrypted_content)


✅ Sample text file 'test.txt' created!
✅ File 'test.txt' encrypted and saved as 'test.txt.enc'!
✅ File 'test.txt.enc' decrypted and saved as 'test.txt_decrypted.txt'!

🔍 Decrypted Content:
This is a simple encryption test. Let's see if it works correctly!


In [ ]:
import heapq
import os
import random
import string
from collections import Counter, namedtuple

# ---------------- STEP 1: Create a Sample Text File ----------------

def create_sample_text_file(filename, text):
    """Creates a text file with given text."""
    with open(filename, 'w') as file:
        file.write(text)
    print(f"✅ Sample text file '{filename}' created!")

# ---------------- STEP 2: Huffman Encoding Functions ----------------

class Node(namedtuple("Node", ["char", "freq", "left", "right"])):
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    """Builds the Huffman tree for encoding."""
    freq = Counter(text)
    heap = [Node(char, freq, None, None) for char, freq in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(None, left.freq + right.freq, left, right)
        heapq.heappush(heap, merged)

    return heap[0]

def generate_huffman_codes(node, prefix="", codebook={}):
    """Generates Huffman encoding dictionary."""
    if node:
        if node.char:
            codebook[node.char] = prefix
        generate_huffman_codes(node.left, prefix + "0", codebook)
        generate_huffman_codes(node.right, prefix + "1", codebook)
    return codebook

# ---------------- STEP 3: Huffman Encrypt (Compress) ----------------

def encrypt_file(input_filename):
    """Encodes a file using Huffman compression."""
    if not os.path.exists(input_filename):
        print(f"❌ Error: File '{input_filename}' not found!")
        return None

    with open(input_filename, 'r') as file:
        text = file.read()

    # Step 1: Build Huffman Tree & Generate Codes
    huffman_tree = build_huffman_tree(text)
    codes = generate_huffman_codes(huffman_tree)

    # Step 2: Encode text using Huffman codes
    encoded_text = ''.join(codes[char] for char in text)

    # Step 3: Save compressed file
    encrypted_filename = input_filename + ".huff"
    with open(encrypted_filename, 'w') as file:
        file.write(encoded_text)

    print(f"✅ File '{input_filename}' compressed and saved as '{encrypted_filename}'!")
    return encrypted_filename, huffman_tree  # Return file & Huffman tree for decoding

# ---------------- STEP 4: Huffman Decrypt (Decompress) ----------------

def decrypt_file(encrypted_filename, huffman_tree):
    """Decodes a Huffman compressed file back to original."""
    if not os.path.exists(encrypted_filename):
        print(f"❌ Error: File '{encrypted_filename}' not found!")
        return None

    with open(encrypted_filename, 'r') as file:
        encoded_text = file.read()

    # Decode Huffman encoding
    decoded_text = []
    node = huffman_tree
    for bit in encoded_text:
        node = node.left if bit == "0" else node.right
        if node.char:
            decoded_text.append(node.char)
            node = huffman_tree

    # Save the decompressed file
    decrypted_filename = encrypted_filename.replace(".huff", "_decompressed.txt")
    with open(decrypted_filename, 'w') as file:
        file.write(''.join(decoded_text))

    print(f"✅ File '{encrypted_filename}' decompressed and saved as '{decrypted_filename}'!")
    return decrypted_filename

# ---------------- RUN HUFFMAN ENCRYPTION & DECRYPTION ----------------

# Step 1: Create a sample text file
sample_text = "This is a simple encryption test. Let's see if Huffman compression works correctly!"
text_filename = "test.txt"
create_sample_text_file(text_filename, sample_text)

# Step 2: Encrypt (Compress) the file using Huffman encoding
encrypted_file, tree = encrypt_file(text_filename)

# Step 3: Decrypt (Decompress) the file using Huffman decoding
if encrypted_file:
    decrypted_file = decrypt_file(encrypted_file, tree)

    # Step 4: Verify if decompression worked
    with open(decrypted_file, 'r') as file:
        decrypted_content = file.read()
    print("\n🔍 Decrypted Content:")
    print(decrypted_content)


✅ Sample text file 'test.txt' created!
✅ File 'test.txt' compressed and saved as 'test.txt.huff'!
✅ File 'test.txt.huff' decompressed and saved as 'test.txt_decompressed.txt'!

🔍 Decrypted Content:
This is a simple encryption test. Let's see if Huffman compression works correctly!


In [1]:
import random
import string
import numpy as np

# ---------------- STEP 1: Generate a 1KB Random Key ----------------
def generate_random_key():
    """Generates a secure 1KB encryption key."""
    encryption_key = ''.join(random.choices(string.ascii_letters + string.digits + "!@#$%^&*", k=1024))
    print(f"DEBUG: Generated Encryption Key:\n{encryption_key}")
    return encryption_key

# ---------------- STEP 2: 3D Cube Randomization Functions ----------------
def reshape_to_cube(data):
    """Reshapes 1D binary data into a 3D cube while ensuring padding is handled correctly."""
    length = len(data)
    print(f"DEBUG: Original Data Length = {length}")

    cube_size = int(round(length ** (1/3)))  # Approximate cube root
    while cube_size**3 < length:
        cube_size += 1  # Ensure cube can hold the entire data

    padded_length = cube_size**3
    padding_needed = padded_length - length
    print(f"DEBUG: Cube Size = {cube_size}, Padding Needed = {padding_needed}")

    # Pad only with zeros and store padding separately
    padded_data = data + '0' * padding_needed
    print(f"DEBUG: Padded Data:\n{padded_data}")

    # Reshape with row-major order ('C') for consistent handling
    cube = np.array(list(padded_data)).reshape((cube_size, cube_size, cube_size), order='C')

    return cube, padding_needed, cube_size

def remove_padding(cube, original_length):
    """Removes padding from the cube after undoing shifts."""
    flattened_data = ''.join(cube.flatten(order='C'))
    unpadded_data = flattened_data[:original_length]  # Keep only the original length
    print(f"DEBUG: Data After Removing Padding:\n{unpadded_data}")
    return unpadded_data

def apply_3d_randomization(data):
    """Converts 1D data to a 3D cube, applies random shifts, and keeps it in 3D format."""
    print(f"DEBUG: Original Data Before 3D Randomization:\n{data}")

    cube, padding, cube_size = reshape_to_cube(data)

    # Store original cube for debugging
    original_cube = np.copy(cube)

    # Generate random shifts for each axis
    shifts = {
        "axis_0": random.randint(1, cube_size - 1),
        "axis_1": random.randint(1, cube_size - 1),
        "axis_2": random.randint(1, cube_size - 1)
    }
    print(f"DEBUG: 3D Shifts Applied: X={shifts['axis_0']}, Y={shifts['axis_1']}, Z={shifts['axis_2']}")

    # Apply shifts in this order: X → Y → Z
    cube = np.roll(cube, shifts["axis_0"], axis=0)
    cube = np.roll(cube, shifts["axis_1"], axis=1)
    cube = np.roll(cube, shifts["axis_2"], axis=2)

    print(f"DEBUG: Cube After Randomization:\n{cube}")

    return cube, shifts, padding, cube_size

def undo_3d_randomization(cube, shifts, original_length):
    """Reverses 3D cube shifts to recover original data correctly, then removes padding."""
    print(f"DEBUG: Cube Before Undoing Shifts:\n{cube}")

    cube_size = cube.shape[0]

    print(f"DEBUG: Undoing 3D Shifts: X={-shifts['axis_0']}, Y={-shifts['axis_1']}, Z={-shifts['axis_2']}")

    # Reverse shifts **in the exact same order they were applied**: Z → Y → X
    cube = np.roll(cube, -shifts["axis_2"], axis=2)  # Undo Z shift
    cube = np.roll(cube, -shifts["axis_1"], axis=1)  # Undo Y shift
    cube = np.roll(cube, -shifts["axis_0"], axis=0)  # Undo X shift

    print(f"DEBUG: Cube After Undoing Shifts:\n{cube}")

    # Remove padding and return original data
    recovered_data = remove_padding(cube, original_length)
    return recovered_data

# ---------------- STEP 3: Generate and Randomize Huffman Data ----------------
def generate_huffman_encoded_data(length=50):
    """Generates a random Huffman-encoded binary string (0s and 1s)."""
    binary_data = ''.join(random.choices("01", k=length))
    print(f"DEBUG: Generated Huffman Encoded Data:\n{binary_data}")
    return binary_data

# ---------------- TESTING THE IMPLEMENTATION ----------------

# Generate and print the 1KB random key
random_key = generate_random_key()
print(f"🔑 Generated 1KB Random Key:\n{random_key}... (truncated)\n")

# Generate random Huffman-encoded binary data
huffman_data = generate_huffman_encoded_data(50)  # 50-bit random Huffman data
print(f"📄 Original Huffman-encoded Data:\n{huffman_data}\n")

# Apply 3D cube randomization
randomized_cube, shifts, padding, cube_size = apply_3d_randomization(huffman_data)

# Undo 3D cube randomization and remove padding
recovered_data = undo_3d_randomization(randomized_cube, shifts, len(huffman_data))

# Verification (Compare final recovered data with the original)
if huffman_data == recovered_data:
    print("✅ 3D Cube Randomization and Undo Process Worked Correctly!")
else:
    print("❌ Something Went Wrong! Check debug output for inconsistencies.")


DEBUG: Generated Encryption Key:
Un$D1JT!LvxwTE&Jo@^TVWN9d2E4DmRmktD0w3Ae9oYT*v%G%9#X&62weMC8B0SnQd%AEiHwI87yN*!xw^1uXtxSqMrZA&$unxV1ImParRu@Us6zlp5Lf$0wQm9rTO$4fEe3DWM6bdOG2^QtqQwP#f@iQUr$6#puYxqw8fiC$*JGUZo$0Kgm$S$0TB@wvUZR$*^ZsFheDAqk%SGopOEpwU0qasmMH*7jhmPpI!yD%#7p$dBkpQgTvbRfNsUfhCGBZUUz5Xnd!wq2RX#qr@Iq!XcOns3$R^ZEV8q9##czd2esYll$2cKA#gSvw1J$Fon4y&teZxNAHqSItSPNKeZrWZbsXrQ0RPlmivbsdn4R!AfaWIr9ev4iKF1gPZmVqn7AOLC7Q*&5EA&pRF6x#dc1sfAAqnmmAc9oZGzQzO1!A1K^yvNkN0**uamTr8UXpDcCTA32JpeVZof6SpJ6Ajbig^kthAF#xeoL1w8MplzdB@k5rSvsy*Z*Y#vpXw9XZ7whLK$MmgmO7JeMOAEZKg4J6!o6m$mtjtQXoYDmSIm!jESXq5Kl!aQacjx63soYV*8SNDojFXUmd!H^0fNa3yxA9Q2IQgZnXgofe4Wa^Sk7TddqLdSoGRR3c9*BRRplkYujSD!x7HlOSdr^5C3v66pQ%4d#R3DHtMuFQ5WLJw$Y4u76AniWK5Y5TEjVjXI6vA!gZP#YPutxjM%z@DpVUniF@QiK4nQFdymD%Q$mV3rk&OGoAl*I5rR5^pBX6GTSUhLhE^TsXeqoHSCg$LG1wuroEBI^EYHeUzpz9anUDZHSqh4zp3FQ4YJQyVB0NGBtMSYjbsEcM6pArKF7yL5#xvYkX*3r&$JnVbfBXpTD&doKw4cAw1mjTx8Q1IUHzI$ztKNYhylf#8oi$S0uLrQG#*p64oCm4zu^UjFIbAyt&HG^pLoTlg#&xLKUfx69DW5fk!kcLqqRB0g